In [1]:
import numpy as np
import pandas as pd
import sqlalchemy as sql
import matplotlib.pyplot as plt
import plotly.express as px
import pycountry
import datetime
from PIL import Image
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator

In [ ]:
sql_engine = sql.create_engine('postgresql:///metadb')

## Compute interference rates for 2019

First, we find all measurements that started in the year 2019. Because the measurement table doesn't associate each measurement with a country code, we join each measurement with its report (in the report table), and group the results by country code. The columns are:
- country_code
- num_confirmed_interference: number of confirmed incidents / country
- num_anomaly: number of anomalous incidents / country

In [ ]:
rates_filename = "interference_rates_2019.csv"

In [ ]:
counts_2019 = pd.read_sql_query("""select probe_cc as country_code, 
    count(case when confirmed then 1 end) as num_confirmed_interference,
    count(case when anomaly then 1 end) as num_anomaly
    count(case when not confirmed then 1 end) as num_no_confirmed_interference
    from (select * from measurement where to_char(measurement.measurement_start_time, 'YYYY') = '2019') meas
    left join (select * from report where to_char(measurement.measurement_start_time, 'YYYY') = '2019') rep
    on meas.report_no = rep.report_no group by probe_cc
""")

In [ ]:
# write the query results intermediately to a csv in case kernel dies
counts_2019.write_csv(rates_filename)

In [ ]:
counts_2019 = pd.read_csv(rates_filename)

We're missing the strict and loose rate columns per country. Note that we calculate <b>strict rate</b> over a date range as:
$$\frac{\text{number of confirmed events}}{\text{total number of events}}$$
Similarly, we calculate <b>loose rate</b> as:
$$\frac{\text{number of confirmed events + number of anomalous events}}{\text{total number of events}}$$

In [ ]:
counts_2019["strict_rate"] = counts_2019["num_confirmed_interference"] / (counts_2019["num_confirmed_interference"] + counts_2019["num_no_confirmed_interference"])
counts_2019["loose_rate"] = (counts_2019["num_confirmed_interference"] + counts_2019["num_anomaly"]) / (counts_2019["num_confirmed_interference"] + counts_2019["num_no_confirmed_interference"])

In [ ]:
# overwrite the intermediate query results with the final table 
counts_2019.write_csv(rates_filename)

Which countries have the highest strict rates over 2019?

In [ ]:
counts_2019.sort_values(by = "strict_rate", ascending = False)

The highest loose rates?

In [ ]:
counts_2019.sort_values(by = "loose_rate", ascending = False)

These tables aren't very readable when it comes to comparing different country rates, so let's make some chloropleth maps to compare them more visually.

In [ ]:
# blue = lower interference rate, red = higher interference rate
blured_color_scheme = px.colors.diverging.RdBu[::-1]

countries = {}
for country in pycountry.countries:
    countries[country.alpha_2] = country.alpha_3
    
counts_2019['iso_alpha3'] = counts_2019['country_code'].apply(lambda x: countries[x] if x in countries else None)

map_rates_2019 = new_rates_2019.copy()
map_rates_2019.head()

In [ ]:
# 2019 map for strict rate
fig = px.choropleth(map_rates_2019,
                    title = "Rate of Confirmed Network Interference Events by Country (2019)",
                    locations = "iso_alpha3",
                    color = "Strict Rate", # strict_rate is a column of new_rates_2019
                    hover_name = "country_code", # column to add to hover information
                    color_continuous_scale = blured_color_scheme)

fig.update_layout(
    title = {
        'y':0.9,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'})

fig.show()

In [ ]:
# 2019 map for loose rate
fig = px.choropleth(map_rates_2019, locations = "iso_alpha3",
                    title = "Rate of Confirmed or Anomalous Network Interference Events by Country (2019)",
                    color = "Loose Rate", # loose_rate is a column of new_rates_2019    
                    hover_name = "country_code", # column to add to hover information
                    color_continuous_scale = blured_color_scheme)

fig.update_layout(
    title = {
        'y':0.9,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'})

fig.show()

From the strict rates map, the most interesting countries are Saudi Arabia, Iran, Russia, Romania, and India in order of decreasing rate.

From the loose rates map, the 5 countries with the highest rates are Ukraine, Iran, Russia, China, and India.

For both strict and loose rates, we'll use the U.S. as a basis for comparison out of familiarity with national events. Let's take a closer look at how both types of interference rates changed by day to get to their final rankings on 12/30/19.

## Interference rates by day

To keep track of how the interference rates change by day, we need to cumulatively sum the number of confirmed, anomalous (and not confirmed) events, grouping by day and country. For example, the cumulative_num_anomaly value for some day $x$ should be the <i>sum</i> of the number of anomalous events for every day in the date range up to and including $x$ for a specific country. Our final table should have the columns below:
- country_code
- date
- cumulative_num_anomaly
- cumulative_num_confirmed
- cumulative_num_not_confirmed
- cumulative_strict_rate
- cumulative_loose_rate

In [ ]:
daily_rates_filename = "interference_rates_by_day_2019.csv"

In [ ]:
daily_rates_2019 = pd.read_sql_query("""select
            probe_cc,
            date,
            sum(day_rates.num_anomaly) as cumulative_num_anomaly,
            sum(day_rates.num_confirmed) as cumulative_num_confirmed,
            sum(day_rates.num_not_confirmed) as cumulative_num_not_confirmed
        from (select
            probe_cc as country_code,
            to_char(measurement.measurement_start_time, 'YYYY') as date,
            count(case when confirmed then 1 end) as num_confirmed,
            count(case when anomaly then 1 end) as num_anomaly
            count(case when not confirmed then 1 end) as num_not_confirmed
        from (select * from measurement where to_char(measurement.measurement_start_time, 'YYYY') = '2019') meas
        left join (select * from report where to_char(measurement.measurement_start_time, 'YYYY') = '2019') rep
        on meas.report_no = rep.report_no group by probe_cc, to_char(measurement.measurement_start_time, 'YYYY')) day_rates
        where date <= day_rates.date
""")

## Confirmed and anomalous domains by country

-- focus on China, Russia, and India, showing how to make the word clouds and going into how the domains that pop up are associated with real-world events / legislation